In [6]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from train.datasets_preparation import DatasetPreparation

dataset_preparation = DatasetPreparation()
dataset_preparation.populate_prepared_datasets()


processing slide /src/workspace/slide-analysis-nn/train/datasets/source/slide_images/Tumor_044.tif
(62829, 140689, 62837, 140692) (62825, 140688, 62841, 140693)
(36052, 153821, 36057, 153827) (36050, 153818, 36059, 153830)


(65099, 137514, 65857, 137879) (64720, 137332, 66236, 138061)
(35765, 153827, 37191, 155129) (35052, 153176, 37904, 155780)
(66937, 158682, 68044, 159973) (66384, 158037, 68597, 160618)
(64954, 145838, 66131, 146838) (64366, 145338, 66719, 147338)
(54889, 165912, 59896, 168367) (52386, 164685, 62399, 169594)
(63819, 145703, 65158, 146786) (63150, 145162, 65827, 147327)
(67425, 143628, 69125, 146046) (66575, 142419, 69975, 147255)
(58023, 133486, 61232, 136791)(60275, 149122, 63203, 152682) (58811, 147342, 64667, 154462)
 (56419, 131834, 62836, 138443)


(63650, 130210, 69199, 137839) (60876, 126396, 71973, 141653)


processing slide /src/workspace/slide-analysis-nn/train/datasets/source/slide_images/Tumor_026.tif
(33395, 129532, 33401, 129550) (33392, 129523, 33404, 129559)
(22283, 124299, 22292, 124304)(56472, 121418, 56473, 121418) (56472, 121418, 56473, 121418)
 (22279, 124297, 22296, 124306)
(35827, 133631, 35842, 133652) (35820, 133621, 35849, 133662)
(71031, 144284, 71036, 144287) (71029, 144283, 71038, 144288)
(56400, 121416, 56560, 121510) (56320, 121369, 56640, 121557)
(56568, 121055, 56723, 121242) (56491, 120962, 56800, 121335)
(56431, 121610, 56544, 121705) (56375, 121563, 56600, 121752)
(35843, 133609, 36122, 133945) (35704, 133441, 36261, 134113)
(74555, 141102, 75541, 142030) (74062, 140638, 76034, 142494)
(60459, 99222, 61223, 100287) (60077, 98690, 61605, 100819)
(73832, 141569, 74168, 141762) (73664, 141473, 74336, 141858)
(70334, 144284, 71312, 144572) (69845, 144140, 71801, 144716)
(73090, 142988, 74234, 143685) (72518, 142640, 74806, 144033)


(92204, 126645, 92226, 126691) (92193, 126622, 92237, 126714)
(91172, 126645, 92595, 130550) (90461, 124693, 93306, 132502)
(59349, 132722, 65371, 138737) (56338, 129715, 68382, 141744)
(75961, 190124, 79832, 194016) (74026, 188178, 81767, 195962)


(55713, 120356, 56672, 121966) (55234, 119551, 57151, 122771)


(59253, 158305, 62778, 164416) (57491, 155250, 64540, 167471)
(79152, 164526, 81573, 166768) (77942, 163405, 82783, 167889)


Path /src/workspace/slide-analysis-nn/train/datasets/source/slide_images/Tumor_026_predicted.tif not exist


cut
179.24632024765015
Train data set percentage = 100.00%
train data distribution: {'Tumor': 0.7570139058306904, 'Background': 0.24298609416930958}
validation data distribution: {}


In [1]:

from train import Train

train = Train()
train.start_training()

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


GPU not found


COMET WARNING: We detected that you are running inside a Ipython/Jupyter notebook environment but we cannot save your notebook source code. Please be sure to have installed comet_ml as a notebook server extension by running:
jupyter comet_ml enable


COMET INFO: 


COMET WARNING: Comet.ml support for Ipython Notebook is limited at the moment, automatic monitoring and stdout capturing is deactivated


...

3/3 [==============================] - 24s 8s/step - loss: 1.1728 - acc: 0.7500


Epoch 2/200


1/3 [=========>....................] - ETA: 11s - loss: 1.1083 - acc: 1.0000

2/3 [===================>..........] - ETA: 5s - loss: 1.2736 - acc: 0.7500 

KeyboardInterrupt: 

In [1]:
from prediction.predict import Predict

predict_example = Predict()
prediction = predict_example.predict_slide('/src/workspace/slide-analysis-nn/train/datasets/source/slide_images/hidden/Tumor_015.tif', area_to_predict=((74000, 74000), (80000, 80000)))
prediction.save_as_asap_annotations(truth_xml_path='/src/workspace/slide-analysis-nn/train/datasets/source/slide_images/hidden/Tumor_015true.xml')
# print(prediction)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


SyntaxError: Please import Comet before importing any keras modules (<string>)